In [ ]:
# 데이터셋 경로 (자신의 로컬 경로에 맞게 수정하세요)
original_dataset_dir = '/path/to/original/train'  # 예: 'dataset/train'
base_dir = '/path/to/small_dataset'  # 예: 'dataset_small'

# 학습(train)/검증(validation) 디렉토리 생성
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# 디렉토리가 없다면 생성
os.makedirs(train_cats_dir, exist_ok=True)
os.makedirs(train_dogs_dir, exist_ok=True)
os.makedirs(validation_cats_dir, exist_ok=True)
os.makedirs(validation_dogs_dir, exist_ok=True)

# 예시: 고양이 이미지를 학습용 1000장, 검증용 500장으로 분리
# 개도 마찬가지
num_train = 1000
num_val = 500

# 기존 이미지 리스트
cat_fnames = [f for f in os.listdir(original_dataset_dir) if f.startswith('cat')]
dog_fnames = [f for f in os.listdir(original_dataset_dir) if f.startswith('dog')]

# 랜덤 셔플 (결과 재현성을 위해 시드 고정)
random.seed(42)
random.shuffle(cat_fnames)
random.shuffle(dog_fnames)

# 고양이 이미지 복사
for i, fname in enumerate(cat_fnames):
    src = os.path.join(original_dataset_dir, fname)
    if i < num_train:
        dst = os.path.join(train_cats_dir, fname)
    elif i < num_train + num_val:
        dst = os.path.join(validation_cats_dir, fname)
    else:
        break
    shutil.copyfile(src, dst)

# 강아지 이미지 복사
for i, fname in enumerate(dog_fnames):
    src = os.path.join(original_dataset_dir, fname)
    if i < num_train:
        dst = os.path.join(train_dogs_dir, fname)
    elif i < num_train + num_val:
        dst = os.path.join(validation_dogs_dir, fname)
    else:
        break
    shutil.copyfile(src, dst)

print('이미지 분할 완료:')
print('학습용 고양이:', len(os.listdir(train_cats_dir)))
print('검증용 고양이:', len(os.listdir(validation_cats_dir)))
print('학습용 강아지:', len(os.listdir(train_dogs_dir)))
print('검증용 강아지:', len(os.listdir(validation_dogs_dir)))


In [ ]:
# 학습용 데이터 증강 설정
train_datagen = ImageDataGenerator(
    rescale=1./255,             # 픽셀 값을 [0,1]로 정규화
    rotation_range=40,          # 최대 40도 회전
    width_shift_range=0.2,      # 좌우 이동 범위
    height_shift_range=0.2,     # 상하 이동 범위
    shear_range=0.2,            # 전단 변환
    zoom_range=0.2,             # 확대/축소
    horizontal_flip=True,       # 좌우 뒤집기
    fill_mode='nearest'         # 빈 공간을 채우는 방식
)

# 검증용 데이터는 증강 없이, 정규화만
validation_datagen = ImageDataGenerator(rescale=1./255)

# 배치 단위로 이미지와 라벨을 불러오는 제너레이터 생성
batch_size = 32
image_size = (150, 150)  # CNN 입력 크기

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,    # 모든 이미지를 (150, 150) 크기로 조정
    batch_size=batch_size,
    class_mode='binary'        # 고양이/강아지 구분이므로 이진 분류
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)


In [ ]:
model = models.Sequential([
    # 첫 번째 컨볼루션 블록
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    
    # 두 번째 컨볼루션 블록
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # 세 번째 컨볼루션 블록
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # 네 번째 컨볼루션 블록
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # 분류를 위한 완전연결(FC) 계층
    layers.Flatten(),
    layers.Dropout(0.5),               # 과적합 방지를 위해 Dropout 적용
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # 이진 분류이므로 sigmoid 사용
])

# 모델 구조 요약 출력
model.summary()


In [ ]:
model.compile(
    loss='binary_crossentropy',        # 이진 분류 손실 함수
    optimizer=optimizers.RMSprop(learning_rate=1e-4),  # 학습률을 1e-4로 설정
    metrics=['accuracy']
)


In [ ]:
# 학습할 에포크 수
epochs = 30

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1, epochs + 1)

# 정확도 그래프
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.title('모델 정확도')
plt.legend()

# 손실 그래프
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.title('모델 손실')
plt.legend()

plt.show()


In [ ]:
final_val_acc = val_acc[-1]
final_val_loss = val_loss[-1]
print(f"최종 검증 정확도: {final_val_acc:.4f}, 최종 검증 손실: {final_val_loss:.4f}")

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# 테스트할 이미지 경로
test_image_path = '/path/to/some_test_image.jpg'  # 고양이 또는 강아지 사진

img = image.load_img(test_image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # 정규화
img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가

prediction = model.predict(img_array)  # 출력값이 0~1 사이의 실수
# 0에 가까우면 고양이, 1에 가까우면 강아지
if prediction < 0.5:
    print(f"모델 예측: 고양이 (확률: {1-prediction[0][0]:.4f})")
else:
    print(f"모델 예측: 강아지 (확률: {prediction[0][0]:.4f})")